## Minimal KerasCV notebook for latent space walking via prompt interpolation

Let's start by installing what we need.

In [ ]:
!pip install tensorflow keras_cv --upgrade --quiet
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

In [ ]:
import keras_cv
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from PIL import Image

# Enable mixed precision
# (only do this if you have a recent NVIDIA GPU)
keras.mixed_precision.set_global_policy("mixed_float16")

# Instantiate the Stable Diffusion model
model = keras_cv.models.StableDiffusion(img_height=512, img_width=512, jit_compile=True)

By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE


Here's a gif saving utility that will come in handy.

In [ ]:
def export_as_gif(filename, images, frames_per_second=13, rubber_band=False):
    if rubber_band:
        images += images[2:-1][::-1]
    images[0].save(
        filename,
        save_all=True,
        append_images=images[1:],
        duration=1000 // frames_per_second,
        loop=0,
    )

## These are our configurable parameters

We will walk from `prompt1` to `prompt2`.

In [ ]:
prompt_1 = "Mysterious ruins in the snow, concept art, digital, artstation"
prompt_2 = "Mysterious ruins in the summer sunlight, concept art, digital, artstation"
interpolation_steps = 50
filename = "ruins_1.gif"

## Now let's generate

In [ ]:
encoding_1 = tf.squeeze(model.encode_text(prompt_1))
encoding_2 = tf.squeeze(model.encode_text(prompt_2))

batch_size = 10 # This value might to be lowered to 3 on a smaller GPU.
batches = interpolation_steps // batch_size

interpolated_encodings = tf.linspace(encoding_1, encoding_2, interpolation_steps)
batched_encodings = tf.split(interpolated_encodings, batches)

seed = 1337
noise = tf.random.normal((512 // 8, 512 // 8, 4), seed=seed)

images = []
for batch in range(batches):
    images += [
        Image.fromarray(img)
        for img in model.generate_image(
            batched_encodings[batch],
            batch_size=batch_size,
            num_steps=25,
            diffusion_noise=noise,
        )
    ]

export_as_gif(filename, images, rubber_band=True)

25/25 [==============================] - 8s 339ms/step


To see the generated gif, look for the file browser tab on your left.